# Nachbarschaft mit RANSAC

In [4]:
import pdal 
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
from scipy.spatial import KDTree
import os
import json
import pyvista as pv
from sklearn.linear_model import RANSACRegressor

from interessant import * # Bei Änderungen Kernel neu starten

In [5]:
run = run24
#run = run14
# filename = interessant['OLA gleiche Höhe wie Gleis']

# Bahnsteig: 29; Gleis hohe Intensität: 11; Weiche B: 16; Unterirdischer Bhf: 20; Gleis weit abseits: 23; Betondeckel: 28; Zug run 14 A (in run24 Achszähler): 6; 
# Viele Gleise: 33; Anfang Weiche: 34; OLA gleiche H: 35; Y: 37
key = list(interessant.keys())[0] 
filename = interessant[key]
print(key, filename)

filename = os.path.join(run, filename)
if not os.path.exists(filename):
    raise FileNotFoundError(filename)

Einfach 4473900_5335875.copc.laz


In [6]:
thresh = 8  # z.B. 5 oder 8
majority_tresh  = 0.5 # Erster Durchgang 0.3, bei "Gleis hohe Intensität" gibt 0.5 ein viel besseres Ergebnis

voxel_size = 1.0

voxel_size = 25 / 30
print("Voxel size:", voxel_size)

minimum_points = 50 # Erste Versuche mit 100, aber viel schwarz bei abseits liegenden Gleisen. 50 ist besser.
minimum_in_hood = 10
linearity_tresh = 0.98

intensity_threshold = 14500
downsample_radius = 0.3
neighborhood_radius = 0.5

Voxel size: 0.8333333333333334


In [7]:
import subprocess
#subprocess.Popen(["pyvistaviewer", filename])

## Voxelfilter

In [8]:
pipeline = pdal.Pipeline([pdal.Reader(filename)])
pipeline.execute()
points = pipeline.arrays[0]

In [9]:
xyz = np.vstack((points['X'], points['Y'], points['Z'])).transpose()

In [10]:
# Offset entfernen (aber gerundet, damit Kachelgrenzen ganze Zahlen bleiben)
offset = xyz.mean(axis=0).round() 
# xyz -= offset   # Nur für Visualisierung benötigt

In [11]:
points['Classification'] = 0 # Unclassified
RAIL = 20

In [12]:
maxp = xyz.max(axis=0)
minp = xyz.min(axis=0)
maxp, minp

(array([4.4739250e+06, 5.3359000e+06, 5.3012408e+02]),
 array([4.4739000e+06, 5.3358750e+06, 5.1300348e+02]))

In [13]:
voxels = xyz.copy()
voxels[:, :2] = ((xyz[:, :2] - minp[:2]) // voxel_size).astype(int)

In [14]:
# Anzahl der Voxel checken
np.ceil((maxp[:2] - minp[:2]) / voxel_size).astype(int)

array([30, 30])

In [15]:
from collections import defaultdict
voxel_dict = defaultdict(list)
index_dict = defaultdict(list)

# Füllen des Dictionaries
for idx, (point, voxel) in enumerate(zip(xyz, voxels)):
    voxel_key = tuple(voxel[:2])
    voxel_dict[voxel_key].append(point[2])
    index_dict[voxel_key].append(idx)

In [16]:
for key, z_values in voxel_dict.items():
    
    # Threshold on number of points in voxel
    if len(z_values) < minimum_points:
        continue

    indices = np.array(index_dict[key])
    z_values = np.array(z_values)
    ground_level = np.percentile(z_values, 10) # 10% Percentile
    # Check that there are almost no points 0.5 to 4.5 m above the ground
    # But allow for some noise
    # thresh = 3 # Der einfachheit halber oben
    count = ((z_values > ground_level + 0.5) & (z_values < ground_level + 4.5)).sum()

    if count <= thresh:
        # Look for points within 0.5 m above ground and get 98% percentile ODER 99.5
        mask = (z_values > ground_level) & (z_values < ground_level + 0.5)
        try:
            candidates_top = np.percentile(z_values[mask], 99.5)
        except IndexError:
            # Fails if there are no points in the masked array
            continue

        # Oude Elberink require the height difference > 0.1 m
        # And mark only the points 10 cm below the top as rail point candidates
        if candidates_top - ground_level > 0.1:
            mask = (z_values > candidates_top - 0.1) & (z_values < candidates_top + 0.05)

            # Also make sure these are only a minority of the points (otherwise it's a slope)
            if mask.sum() < majority_tresh * len(z_values):  # z.B. 0.3
                points['Classification'][indices[mask]] = RAIL


In [17]:
candidates = points[points["Classification"] == RAIL]
candidates.shape

(92692,)

## Noise Filter

In [18]:
# filters.outlier sets Classification to 7, filters.range removes the points with Classification 7

noise_filter = pdal.Filter("filters.outlier", method="statistical", mean_k=10, multiplier=2.0).pipeline(candidates) | pdal.Filter("filters.range", limits="Classification![7:7]")
print(noise_filter.toJSON())
noise_filter.execute()
candidates = noise_filter.arrays[0]
candidates.shape 

[{"type": "filters.outlier", "method": "statistical", "mean_k": 10, "multiplier": 2.0, "tag": "filters_outlier1"}, {"type": "filters.range", "limits": "Classification![7:7]", "tag": "filters_range1"}]


(91051,)

## View Settings

In [19]:
# Viewsettings mit strg + c kopieren und hier einfügen

viewsettings = '''
{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 11.999975427985191, 11.99998692702502, 13.124079998226534 ],
			"boundingbox_min" : [ -13.000024572014809, -13.00001307297498, -3.9965200017734333 ],
			"field_of_view" : 60.0,
			"front" : [ -0.20468464372193082, -0.82045900926496551, 0.53380821531742795 ],
			"lookat" : [ -2.1145501200370735, -2.6052610037108783, 1.4494799802055294 ],
			"up" : [ 0.19010212482081987, 0.50164960558000959, 0.84392467398461002 ],
			"zoom" : 0.55999999999999983
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}

'''

viewsettings = json.loads(viewsettings)

front = viewsettings["trajectory"][0]["front"]
lookat = viewsettings["trajectory"][0]["lookat"]
up = viewsettings["trajectory"][0]["up"]
zoom = viewsettings["trajectory"][0]["zoom"]

## Candidate and Seed Points

In [20]:
xyz = np.vstack((candidates['X'], candidates['Y'], candidates['Z'])).transpose()
xyz -= offset

In [21]:
low_intensity = candidates[candidates["Intensity"] < intensity_threshold]
low_intensity.shape

(49162,)

In [22]:
# xyz_low = np.vstack((low_intensity['X'], low_intensity['Y'], low_intensity['Z'])).transpose()
# xyz_low -= offset

# pcd_low_intensity = o3d.geometry.PointCloud()
# pcd_low_intensity.points = o3d.utility.Vector3dVector(xyz_low)
# pcd_low_intensity.paint_uniform_color([0, 0, 0.7])

In [23]:
# Downsample with poisson sampling

downsampling_pipeline = pdal.Filter("filters.sample", radius=downsample_radius).pipeline(low_intensity)
downsampling_pipeline.execute()
seed_points = downsampling_pipeline.arrays[0]
seed_points.shape 

(396,)

In [24]:
xyz_seed = np.vstack((seed_points['X'], seed_points['Y'], seed_points['Z'])).transpose()
xyz_seed -= offset

# pcd_seed_points = o3d.geometry.PointCloud()
# pcd_seed_points.points = o3d.utility.Vector3dVector(xyz_seed)
# pcd_seed_points.paint_uniform_color([1, 0, 0])

In [25]:
# o3d.visualization.draw_geometries([
#     pcd_candidates, 
#     pcd_low_intensity, 
#     pcd_seed_points
#     ], front=front, lookat=lookat, up=up, zoom=zoom)

In [26]:
# o3d.visualization.draw_geometries([pcd_seed_points], front=front, lookat=lookat, up=up, zoom=zoom)

In [27]:
# k-D tree with all candidate points
tree = KDTree(xyz)  

In [28]:
# indices: ndarray (dtype object) with a list of indices for each seed point
indices = tree.query_ball_point(xyz_seed, r=neighborhood_radius)

In [29]:
seed_point_count = xyz_seed.shape[0]

In [30]:
def pca(cloud):
    """Use PCA to get einvalues and eigenvectors of a point cloud"""
    mean = np.mean(cloud, axis=0)
    centered = cloud - mean
    cov_matrix = np.cov(centered, rowvar=False) # row variance nicht berechnen
    eigenvals, eigenvecs = np.linalg.eig(cov_matrix)
    sorted_indices = np.argsort(eigenvals)[::-1]
    sorted_eigenvals = eigenvals[sorted_indices]
    sorted_eigenvecs = eigenvecs[:,sorted_indices]
    return sorted_eigenvals, sorted_eigenvecs

def linearity(eigenvals):
    """Calculate the linearity of a point cloud"""
    return (eigenvals[0] - eigenvals[1]) / eigenvals[0]

def planarity(eigenvals):
    """Calculate the planarity of a point cloud"""
    return (eigenvals[1] - eigenvals[2]) / eigenvals[0]

In [31]:
def theta(eigenvects):
    """Angle between the first eigenvector and the z-axis"""
    cos_theta = eigenvects.T[0] @ np.array([0, 0, 1]) / np.linalg.norm(eigenvects[0])
    return np.arccos(cos_theta) * 180 / np.pi

In [32]:
def fit_line_ransac(points, threshold=0.1, min_samples=20):
    """Fits a line to the given 2D points using RANSAC."""
    if len(points) < min_samples:
        return None, None  # Not enough points

    ransac = RANSACRegressor(residual_threshold=threshold)
    ransac.fit(points[:, [0]], points[:, 1])  # X -> Y mapping
    
    inlier_mask = ransac.inlier_mask_
    a, b = ransac.estimator_.coef_[0], ransac.estimator_.intercept_
    
    return (a, b), inlier_mask  # Line equation and inliers

In [33]:
i = 15

hood = xyz[indices[i]]
if hood.shape[0] < minimum_in_hood:   
    raise ValueError("Not enough points in neighborhood") # continue
print(f"Seed point {i} has {hood.shape[0]} points in its neighborhood")

eigenvals, eigenvecs = pca(hood)
print("Planarity:" , planarity(eigenvals))
print("Linearity:", linearity(eigenvals))
print("Eigenvals", eigenvals)
print("Theta:", theta(eigenvecs))

hood_2d = hood[:, :2]

model1, inliers1 = fit_line_ransac(hood_2d)


if model1 is not None:
    print("Model 1", model1, "Inliers", inliers1.sum())
    rest = hood_2d[~inliers1]
    model2, inliers2 = fit_line_ransac(rest)
else:
    model2 = None



colors = np.ones((hood.shape[0], 3), dtype=np.float64) 
colors = colors * 0.5 # Gray

if model1 is not None:
    colors[inliers1] = [1, 0, 0]  # Red
else:
    print("No model")
if model2 is not None:
    print("Model 2", model2, "Inliers", inliers2.sum())
    # Inliers2 has the shape of the remains of RANSAC pass 1
    # Map it back to the original indices
    inliers2_full = np.zeros(hood.shape[0], dtype=bool)
    inliers2_full[~inliers1] = inliers2
    colors[inliers2_full] = [0, 1, 0]  # Green


hood_pcd = o3d.geometry.PointCloud()
hood_pcd.points = o3d.utility.Vector3dVector(hood)
hood_pcd.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([hood_pcd])



Seed point 15 has 405 points in its neighborhood
Planarity: 0.0017689527885354872
Linearity: 0.9936550207338047
Eigenvals [0.08174322 0.00051866 0.00037406]
Theta: 89.69895390546952
Model 1 (np.float64(-5.5009657747768985), np.float64(0.8841758228134708)) Inliers 177
Model 2 (np.float64(-5.628164695315011), np.float64(0.7436807315963441)) Inliers 109
